In [5]:
## Import the required libraries 

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from keras.utils.data_utils import GeneratorEnqueuer
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import math, os

%matplotlib inline

Using TensorFlow backend.
/anaconda3/envs/p3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/envs/p3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load images and create image generator

In [6]:
image_path = "./images/"

batch_size = 100
img_generator = ImageDataGenerator().flow_from_directory(image_path, shuffle=False, batch_size = batch_size)
n_rounds = math.ceil(img_generator.samples / img_generator.batch_size)
filenames = img_generator.filenames

img_generator = GeneratorEnqueuer(img_generator)
img_generator.start()
img_generator = img_generator.get()

Found 6 images belonging to 2 classes.


# Map RetinaNet classes to Google object detection classes

In [71]:
# Read classes from RetinaNet. Prepared list taken from: https://github.com/fizyr/keras-retinanet/blob/master/examples/ResNet50RetinaNet.ipynb
retina_classes = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
retina_classes = pd.DataFrame(list(labels_to_names.items()), columns=['ID', 'Label'])
retina_classes.head()


,ID,Label
0,0,person
1,1,bicycle
2,2,car
3,3,motorcycle
4,4,airplane


In [75]:
# Read classes from Google
google_classes = pd.read_csv("class-descriptions-boxable.csv", delimiter=";", names=['ID', 'Label'])
google_classes.head()

,ID,Label
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [76]:
# Transform Google classes
google_classes_lower = google_classes['Label'].str.lower()
google_classes = pd.DataFrame([google_classes["ID"], google_classes_lower]).transpose()
google_classes.head()

,ID,Label
0,/m/011k07,tortoise
1,/m/011q46kg,container
2,/m/012074,magpie
3,/m/0120dh,sea turtle
4,/m/01226z,football


In [ ]:
# Use class key mappings for predictions

In [92]:
# Iterate over Retina_classes and check if they're in Google classes.
# If so, take google ID in new class mappings 

label_mappings = pd.DataFrame(columns=['ID', 'Label'])

for i in range(0, retina_classes.shape[0]):
    for j in range(0, google_classes.shape[0]):
        if(retina_classes["Label"][i] == google_classes["Label"][j]):
            label_mappings.loc[i] = google_classes.iloc[j]


# Import imageai RetinaNet model

In [3]:
from imageai.Detection import ObjectDetection
model_weight_path = "resnet50_coco_best_v2.0.1.h5"

execution_path = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath(model_weight_path)
detector.loadModel()

# Predict single image

In [4]:
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "./images/test/0a0a615629231821.jpg"), output_image_path=os.path.join(execution_path , "./images/output/0a0a615629231821new.jpg"), minimum_percentage_probability=50)

# Output detections

In [5]:
for eachObject in detections:
    print(str(eachObject["name"]) + " " + str(eachObject["percentage_probability"]) + " " + str(eachObject["box_points"][0]) + " " + str(eachObject["box_points"][1]) + " " + str(eachObject["box_points"][2]) + " " + str(eachObject["box_points"][3]))

handbag 52.48710513114929 318 580 378 677
person 60.40759086608887 277 269 352 417
person 50.98159909248352 937 287 1016 448
person 53.549426794052124 5 274 110 536
person 53.045231103897095 30 270 158 513
person 80.6997537612915 750 233 984 673
person 58.23795795440674 635 264 783 672
person 71.70459628105164 267 348 396 662
person 82.73561000823975 365 240 583 669
motorcycle 51.68589949607849 3 446 178 667
